In [1]:
!git clone https://github.com/herm23/Project-NNDL.git

Cloning into 'Project-NNDL'...
remote: Enumerating objects: 63, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 63 (delta 13), reused 58 (delta 11), pack-reused 0 (from 0)
Receiving objects: 100% (63/63), 3.19 MiB | 7.32 MiB/s, done.
Resolving deltas: 100% (13/13), done.


In [2]:
%cd Project-NNDL/

/content/Project-NNDL


Link per il datset di CompCar (trovato su sito ufficiale): https://drive.google.com/drive/folders/18EunmjOJsbE5Lh9zA0cZ4wKV6Um46dkg


In [3]:
!pip install -U -q PyDrive

Import necessari

In [4]:
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os

Scarico i dati direttamente dal drive pubblico

In [5]:
#Autenticazione
auth.authenticate_user()

gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [6]:
# Estrazione dei dati dalla cartella
# ID della cartella pubblica
folder_id = '18EunmjOJsbE5Lh9zA0cZ4wKV6Um46dkg'

# Cartella locale per salvare i dati
save_path = '/content/Project-NNDL/compcar_dataset'
os.makedirs(save_path, exist_ok=True)

# Lista file nella cartella Drive
file_list = drive.ListFile({'q': f"'{folder_id}' in parents and trashed=false"}).GetList()

# Scarica tutti i file
for file in file_list:
    print(f"Scaricando {file['title']}")
    file.GetContentFile(os.path.join(save_path, file['title']))



Scaricando sv_data.zip
Scaricando sv_data.z03
Scaricando sv_data.z02
Scaricando sv_data.z01
Scaricando data.zip
Scaricando data.z22


KeyboardInterrupt: 

In [7]:
!apt-get install -y p7zip-full


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-8).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [8]:
%cd /content/Project-NNDL/compcar_dataset

/content/Project-NNDL/compcar_dataset


In [9]:
!7z x data.zip -o./compcar_dataset/unzipped


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 391417395 bytes (374 MiB)

Extracting archive: data.zip
  1% 4096 Open               51% 172032 Open                 86% 286720 Open                --
Path = data.zip
Type = zip
Physical Size = 391417395
Embedded Stub Size = 4
64-bit = +
Total Physical Size = 16539487795
Multivolume = +
Volume Index = 22
Volumes = 23

  0%    
Enter password (will not be echoed):
  0% 5 - data/image/1/1101/2011/07b90decb92ba6.jpg                                                    0% 121 - data/image/1/1109/2011/f8eb2e897646df.jpg

**Creazione del dataset (seguendo modello prof lavoratorio 4)**

In [59]:
import os
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image

class CompCarsDataset(Dataset):
    def __init__(self, root_dir, txt_path=None, inputs_list=None, transform=None):
        self.transform = transform
        self.root_dir = root_dir
        #Lettura immagini
        if txt_path is not None:
          with open(txt_path, 'r') as f:
              self.images = [line.strip() for line in f.readlines()]
        else:
          self.images = inputs_list

        #Corrispondendi marche
        self.makes_list = [p.split('/')[0] for p in self.images]

        #dizionario usato per associare ad ogni marca un indice
        #l'indice è necessario per la loss di Cross Entropy
        self.make_dict = {make: idx for idx, make in enumerate(sorted(set(self.makes_list)))}

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        # print("Hai richiamato get item")
        full_path = os.path.join(self.root_dir, self.images[idx])
        image = Image.open(full_path).convert('RGB')
        make = self.make_dict[self.makes_list[idx]] #estraggo prima il nome del modello e poi ritorno il suo indice corrispondente

        if self.transform:
            image = self.transform(image)

        return image,make

Shufflue degli elementi passati dal path

In [60]:
from sklearn.model_selection import train_test_split

def shuffleElems(path):
  with open(path, 'r') as f:
    items = [line.strip() for line in f.readlines()]

  train_data, val_data = train_test_split(items, test_size=0.2, random_state=42)

  return train_data, val_data

**Divisione di train e test basandosi sul file presente nella cartella train_test_split
**

In [61]:
from torch.utils.data import DataLoader, random_split
from torch.utils.data import Subset
from torchvision.transforms import Compose, Resize, RandomAffine, RandomHorizontalFlip, ToTensor
import torch
import os
import pandas as pd
import h5py
import random
import numpy as np
import matplotlib.pyplot as plt


# Percorsi
root_dir = "/content/Project-NNDL/compcar_dataset/compcar_dataset/unzipped/data/image"
train_txt = "/content/Project-NNDL/compcar_dataset/compcar_dataset/unzipped/data/train_test_split/classification/train.txt"
test_txt = "/content/Project-NNDL/compcar_dataset/compcar_dataset/unzipped/data/train_test_split/classification/test.txt"
train_perc = 0.8
batch_size = 32

# Rendo invariandt a modifiche
transform = Compose([
    Resize((224, 224)),
    RandomAffine(degrees=5),
    RandomHorizontalFlip(),
    ToTensor()
])

# Dataset (effettuo questa operazione perche mi serve avere il validation e training separati)
# Supponiamo che tu abbia il dataset CompCarsDataset

train_list, valid_list = shuffleElems(train_txt)

# print("Primi 10 elementi di train: ", train_list[:10])
# print("Primi 10 elementi di valid: ", valid_list[:10])

train_dataset = CompCarsDataset(root_dir, None, train_list, transform=transform)
valid_dataset = CompCarsDataset(root_dir, None, valid_list, transform=transform)
test_dataset = CompCarsDataset(root_dir, test_txt, None, transform=transform)

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# print di controllo
# print(train_loader[0])

# Rete Resnet 50

Definizione dei blocchi

In [62]:
from torch.nn import Module, Sequential, Conv2d, BatchNorm2d, ReLU

class MainPath(Module):

    def __init__(self, in_channels, filters, kernel_size, stride=1):
        super().__init__()
        F1, F2, F3 = filters
        self.main_path = Sequential(
            Conv2d(in_channels, F1, kernel_size=1, stride=stride),
            BatchNorm2d(F1),
            ReLU(),
            Conv2d(F1, F2, kernel_size=kernel_size, padding=kernel_size//2),
            BatchNorm2d(F2),
            ReLU(),
            Conv2d(F2, F3, kernel_size=1),
            BatchNorm2d(F3),
        )
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, torch.nn.Linear):
            torch.nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                module.bias.data.zero_()
        if isinstance(module, torch.nn.Conv2d):
            torch.nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                module.bias.data.zero_()

    def forward(self, x):
        y = self.main_path(x)
        return y

class IdentityBlock(MainPath):

    def __init__(self, in_channels, filters, kernel_size):
        super().__init__(in_channels, filters, kernel_size)
        self.relu = ReLU()

    def forward(self, x):
        y = self.relu(self.main_path(x) + x)
        return y

In [63]:
class ConvolutionalBlock(MainPath):

    def __init__(self, in_channels, filters, kernel_size):
        super().__init__(in_channels, filters, kernel_size, stride=2)
        self.relu = ReLU()
        self.shortcut_path = Sequential(
            Conv2d(in_channels, filters[2], kernel_size=1, stride=2),
            BatchNorm2d(filters[2])
        )
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, torch.nn.Linear):
            torch.nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                module.bias.data.zero_()
        if isinstance(module, torch.nn.Conv2d):
            torch.nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                module.bias.data.zero_()

    def forward(self, x):
        y = self.relu(self.main_path(x) + self.shortcut_path(x))
        return y

Definizione rete

In [64]:
from torch.nn import MaxPool2d, AvgPool2d, Linear, Dropout
class ResNet50(Module):

    def __init__(self):
        super().__init__()
        self.network = Sequential(
            Conv2d(3, 64, kernel_size=7, stride=2),
            BatchNorm2d(64),
            MaxPool2d(kernel_size=3, stride=2),
            ConvolutionalBlock(64, [64, 64, 256], kernel_size=3),
            Dropout(0.2),
            IdentityBlock(256, [64, 64, 256], kernel_size=3),
            IdentityBlock(256, [64, 64, 256], kernel_size=3),
            ConvolutionalBlock(256, [128, 128, 512], kernel_size=3),
            Dropout(0.2),
            IdentityBlock(512, [128, 128, 512], kernel_size=3),
            IdentityBlock(512, [128, 128, 512], kernel_size=3),
            IdentityBlock(512, [128, 128, 512], kernel_size=3),
            ConvolutionalBlock(512, [256, 256, 1024], kernel_size=3),
            Dropout(0.2),
            IdentityBlock(1024, [256, 256, 1024], kernel_size=3),
            IdentityBlock(1024, [256, 256, 1024], kernel_size=3),
            IdentityBlock(1024, [256, 256, 1024], kernel_size=3),
            IdentityBlock(1024, [256, 256, 1024], kernel_size=3),
            IdentityBlock(1024, [256, 256, 1024], kernel_size=3),
            ConvolutionalBlock(1024, [512, 512, 2048], kernel_size=3),
            Dropout(0.2),
            IdentityBlock(2048, [512, 512, 2048], kernel_size=3),
            IdentityBlock(2048, [512, 512, 2048], kernel_size=3),
            AvgPool2d(kernel_size=2, stride=2)
        )
        # self.classification_layer = Linear(2048, 1)
        self.classification_layer = Linear(8192, 431) #431 per ora è hard coded, lo devi modificare leggendo la lunghezza delle classi totali !!
        self.apply(self._init_weights)

    def forward(self, x):
        y = self.network(x).reshape((x.shape[0], -1))
        # print("shape dopo la resnet 50:", y.shape) # Add this line to print the shape
        y = self.classification_layer(y)
        return y

    def _init_weights(self, module):
        if isinstance(module, torch.nn.Linear):
            torch.nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                module.bias.data.zero_()
        if isinstance(module, torch.nn.Conv2d):
            torch.nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                module.bias.data.zero_()

Train + valid

In [65]:
from torch.optim import SGD, Adam
from torch.nn import CrossEntropyLoss
from tqdm import tqdm
model = ResNet50()
# opt = SGD(model.parameters(), lr=1e-2, weight_decay = 0)
opt = Adam(model.parameters(), lr=1e-2, weight_decay=0)
loss_fn = CrossEntropyLoss()
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
epochs=30
best_val = np.inf
for epoch in range(epochs):
    model.train()
    print(f"Epoch: {epoch+1}")
    iterator = tqdm(train_loader)
    for batch_x, batch_y in iterator:
        batch_x = batch_x.to(device)
        print("stampa batch_y: ", batch_y)
        batch_y = batch_y.to(device)

        y_pred = model(batch_x)

        loss = loss_fn(y_pred, batch_y)

        opt.zero_grad()
        loss.backward()
        opt.step()
        iterator.set_description(f"Train loss: {loss.detach().cpu().numpy()}")

    model.eval()
    with torch.no_grad():
        predictions = []
        true = []
        for batch_x, batch_y in tqdm(val_loader):
            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)

            y_pred = model(batch_x)

            predictions.append(y_pred)
            true.append(batch_y)
        predictions = torch.cat(predictions, axis=0)
        true = torch.cat(true, axis=0)
        val_loss = loss_fn(predictions, true)
        val_acc = (torch.sigmoid(predictions).round() == true).float().mean()
        print(f"loss: {val_loss}, accuracy: {val_acc}")

    if val_loss < best_val:
        print("Saved Model")
        torch.save(model.state_dict(), "model.pt")
        best_val = val_loss



Epoch: 1


  0%|          | 0/401 [00:00<?, ?it/s]

stampa batch_y:  tensor([59,  5, 27, 66, 34, 73, 50, 63, 22, 51, 74,  3, 56, 59, 63, 63,  1, 59,
        65, 50, 65, 35,  1,  9, 39, 59, 73, 51, 60,  0, 27, 65])
shape dopo la resnet 50: torch.Size([32, 8192])


Train loss: 7.259567737579346:   0%|          | 1/401 [00:08<53:46,  8.07s/it]

stampa batch_y:  tensor([74, 56, 65, 74, 43, 31,  1, 25,  1, 13, 31, 59, 31, 71, 59, 63, 59, 29,
        59, 25, 71, 65,  3, 29, 73, 63,  5, 36,  7, 36, 50, 74])
shape dopo la resnet 50: torch.Size([32, 8192])


...

stampa batch_y:  tensor([63, 63, 26,  0, 65, 74, 63, 15, 28, 28, 27,  5, 73, 63, 73, 30, 61,  0,
        60,  9, 62, 22, 33, 58, 30, 50, 59, 74, 15, 59, 63, 27])
shape dopo la resnet 50: torch.Size([32, 8192])


Train loss: 5.139983177185059:   9%|▊         | 35/401 [04:09<42:22,  6.95s/it]

stampa batch_y:  tensor([32, 33, 13, 43, 29, 15, 42, 65, 29, 24, 73, 65, 29, 22, 15, 24, 11,  2,
        62, 59, 59, 39, 16, 59, 73, 59, 65, 63, 13, 13, 65, 27])
shape dopo la resnet 50: torch.Size([32, 8192])


Train loss: 8.1735200881958:   9%|▉         | 36/401 [04:16<43:17,  7.12s/it]


KeyboardInterrupt: 

Valutazione rete

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_curve, roc_auc_score

def evaluate_network(dataloader, model, data_split):
    model.eval()
    with torch.no_grad():
        predictions = []
        true = []
        for batch_x, batch_y in tqdm(dataloader):
            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)

            y_pred = model(batch_x)

            predictions.append(y_pred)
            true.append(batch_y)
        predictions = torch.cat(predictions, axis=0)
        true = torch.cat(true, axis=0)
        loss = loss_fn(predictions, true).detach().cpu().numpy()
        predictions = torch.sigmoid(predictions).detach().cpu().numpy()
        true = true.detach().cpu().numpy()

        fpr, tpr, thresholds = roc_curve(true, predictions)
        auc = roc_auc_score(true, predictions)
        predictions = predictions.round()
        precision, recall, fscore, _= precision_recall_fscore_support(true, predictions, average='binary')
        accuracy = accuracy_score(true, predictions)

        print(f"{data_split} loss: {loss}, accuracy: {accuracy}, precision: {precision}, recall: {recall}, f1: {fscore}, roc_auc: {auc}")

        plt.figure()
        plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % auc)
        plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title(f'{data_split} receiver operating characteristic (ROC)')
        plt.legend(loc="lower right")

In [ ]:
model.load_state_dict(torch.load("model.pt"))
evaluate_network(train_loader, model, "Training Dataset")
evaluate_network(val_loader, model, "Validation Dataset")
evaluate_network(test_loader, model, "Test Dataset")